In [1]:
#Flickr key: cff83b39da75cd79897386706ffd4ba0
#Secret: b915512a46e9ae79

Associate tags from flickr with a picture's id. We will have a book's id from getPics. We can also use getPics to get sentences for a given picture. Need to make a helper function to find important words from a page and also probably include important words for a book for a picture as well.

In [6]:
import flickrapi
import pickle as pk
import requests
import csv
import json
import time

text = pk.load(open('thr-2016-10-5-text.pkl','rb'))
textID = list(set(text.id))

In [7]:
api_key = u'cff83b39da75cd79897386706ffd4ba0'
api_secret = u'b915512a46e9ae79'

flickr = flickrapi.FlickrAPI(api_key, api_secret, format = 'parsed-json')

In [22]:
test = flickr.photos.getInfo(photo_id = 10284323776)
#test['photo']['tags'][then a list of dicts here]
test['photo']['title']['_content'].replace('"', '').strip().split()
for i in range(len(test['photo']['tags']['tag'])):
    print(test['photo']['tags']['tag'][i]['_content'])
flickr.tags.getRelated(tag = 'halloween')
#Maybe we take the only the related tags that appear in n or more (or all) of the tags from getRelated

agriculture
autumn
britishcolumbia
canada
cultivated
fall
farm
field
fruit
gardening
gourd
grass
halloween
harvest
horizontal
jack
landscape
landscaped
lantern
leaf
many
mapleridge
nature
nobody
o
orange
outdoor
outside
patch
pumpkin
pumpkinpatch
pumpkins
ripe
seasonal
squash
stem
sunstar
sunrise
sunset
trees
vegetable
vibrant
wintersquash
yellow


{'stat': 'ok',
 'tags': {'source': 'halloween',
  'tag': [{'_content': 'pumpkin'},
   {'_content': 'costume'},
   {'_content': 'orange'},
   {'_content': 'fall'},
   {'_content': 'scary'},
   {'_content': 'autumn'},
   {'_content': 'pumpkins'},
   {'_content': 'october'},
   {'_content': 'party'},
   {'_content': 'black'},
   {'_content': 'spooky'},
   {'_content': 'cat'},
   {'_content': 'costumes'},
   {'_content': 'fun'},
   {'_content': 'girl'},
   {'_content': 'green'},
   {'_content': 'night'},
   {'_content': 'creepy'},
   {'_content': 'jackolantern'},
   {'_content': 'kids'},
   {'_content': 'portrait'},
   {'_content': 'leaves'},
   {'_content': 'light'},
   {'_content': 'dark'},
   {'_content': 'witch'},
   {'_content': 'ghost'},
   {'_content': 'skull'},
   {'_content': 'blood'},
   {'_content': 'dog'},
   {'_content': 'friends'},
   {'_content': 'trickortreat'},
   {'_content': 'zombie'},
   {'_content': 'woman'},
   {'_content': 'carving'},
   {'_content': 'candle'},
   {'

In [5]:
#Note that the actual tags are accessed via a dict starting with tags then tag, which returns a list of dicts
#Loop through the list and access the tags via _content
''''tags': {'tag': [{'_content': 'chopwellwoods',
     'author': '41888973@N00',
     'authorname': 'Sam Judson',
     'id': '78188-291793938-5385467',
     'machine_tag': 0,
     'raw': 'chopwellwoods'}'''

"'tags': {'tag': [{'_content': 'chopwellwoods',\n     'author': '41888973@N00',\n     'authorname': 'Sam Judson',\n     'id': '78188-291793938-5385467',\n     'machine_tag': 0,\n     'raw': 'chopwellwoods'}"

In [8]:
url = "/cache/images/19/14592458919_f88abe90f9_t.jpg"
first = url.split("/")
second = first[-1].split("_")[0]
second = int(second)
anotherTest = flickr.photos.getInfo(photo_id = second)
print(second)
print(first)

14592458919
['', 'cache', 'images', '19', '14592458919_f88abe90f9_t.jpg']


In [33]:
def getRes(url):
    res = requests.get(url)
    if res.status_code == 200:
        data = res.json()
        return data

def pics(ids):
       
    new_dict = {} #new format dict
    for line in ids: #line is book's id
        address = 'http://gbserver3.cs.unc.edu/book-as-json/?id=' + str(line)
        data = getRes(address) #return json of book details
        
        for i in range(len(data['pages'])): #loop through each book page to get pic ids
            url = data['pages'][i]['url']
            photo = url.split("/")
            thr_photo = photo[-1] #id used by THR for given pic; ends in .jpg
            photo = photo[-1].split("_")[0] #this is id used in Flickr API
            
            new_dict[thr_photo] = {} #each thr photo id gets a dict w/ 'tags', 'title', 'book_id', 'flickr_id'
            new_dict[thr_photo]['flickr_id'] = photo
            new_dict[thr_photo]['tags'] = []
            new_dict[thr_photo]['book_id'] = line
            
            try:
                f_info = flickr.photos.getInfo(photo_id = photo)
            except:
                break
            title = f_info['photo']['title']['_content'].replace('"', '').strip().split()
            
            new_dict[thr_photo]['title'] = title
            tag_info = f_info['photo']['tags'] #stores a list of dicts with '_content' as key we want
            if len(tag_info) != 0:
                
                for k in range(len(tag_info)):
                    
                    tag = tag_info['tag'][k]['_content']
                    related_tags = None
                                                                    
                    if tag not in new_dict[thr_photo]['tags']:
                        new_dict[thr_photo]['tags'].append(tag)
            #time.sleep(0.6)
    return new_dict

In [34]:
textID = textID[:10]
new = pics(textID)
new

{'10128938316_74e4b138f6.jpg': {'book_id': 131079,
  'flickr_id': '10128938316',
  'tags': ['orange'],
  'title': ['pumpkins']},
 '10128938316_74e4b138f6_t.jpg': {'book_id': 131079,
  'flickr_id': '10128938316',
  'tags': ['orange'],
  'title': ['pumpkins']},
 '10284323776_db52749c8a.jpg': {'book_id': 131079,
  'flickr_id': '10284323776',
  'tags': ['agriculture'],
  'title': ['Pumpkin', 'Patch']},
 '1035315307_1a0c6fbef5.jpg': {'book_id': 131079,
  'flickr_id': '1035315307',
  'tags': ['pumpkin'],
  'title': ['Pumpkin', "'Fairytale'"]},
 '10584676434_21b6c338bb.jpg': {'book_id': 131083,
  'flickr_id': '10584676434',
  'tags': ['sad'],
  'title': ['Sad', 'Reese']},
 '10606030685_31224928e8.jpg': {'book_id': 131074,
  'flickr_id': '10606030685',
  'tags': ['giraffe'],
  'title': ['Giraffe', 'landscape']},
 '11399067203_1d1238a9a3.jpg': {'book_id': 131081,
  'flickr_id': '11399067203',
  'tags': ['spoon'],
  'title': ['Spoon']},
 '11399067203_1d1238a9a3_t.jpg': {'book_id': 131081,
  'fli